# Setup

In [1]:
import numpy as np
import pandas as pd

In [2]:
def display_df(df, n=1, title=None):
    if title:
        print(title + ':')
    display(df.head(n), df.tail(n), df.shape)

In [3]:
data_raw_dir = '../data/raw/'
data_interim_dir = '../data/interim/'

# Read and Process Datasets

In [4]:
def process_seeds(df):
    """Process tournament seed file"""
    return (
        df
        .assign(Seed=df.Seed.map(lambda s: int(s[1:3])))
        .assign(IsPlayIn=df.Seed.map(lambda s: str(s).endswith('a') or str(s).endswith('b')))
    )


def process_games(df):
    """Process tournament game file"""
    
    def _standardize_team_ID(row):
        """TBD"""
        if row.WTeamID < row.LTeamID:
            row.TeamOneID = row.WTeamID
            row.TeamTwoID = row.LTeamID
        else:
            row.TeamOneID = row.LTeamID
            row.TeamTwoID = row.WTeamID
        return row

    def _create_labels(row):
        """TBD"""
        if row.TeamOneID == row.WTeamID:
            row.Label = 1
        else:
            row.Label = 0
        return row
    
    return (
        df
        .assign(
            TeamOneID=np.nan,
            TeamTwoID=np.nan,
            Label=np.nan
        )
        .apply(_standardize_team_ID, axis=1)
        .apply(_create_labels, axis=1)
        .drop(['WTeamID', 'LTeamID', 'DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
    )


def process_submission_games(df):
    """Process tournament game submission file"""
    return (
        df
        .assign(
            Season=df.ID.str.split('_', expand=True)[0].astype(int),    
            TeamOneID=df.ID.str.split('_', expand=True)[1].astype(int),
            TeamTwoID=df.ID.str.split('_', expand=True)[2].astype(int),
    )
    .drop(['ID', 'Pred'], axis=1)
)

In [5]:
# Read in seeds
df_seeds = (
    pd.read_csv(data_raw_dir + 'DataFiles/NCAATourneySeeds.csv')
    .pipe(process_seeds)
)

# Read in tournament games
df_games = (
    pd.read_csv(data_raw_dir + 'DataFiles/NCAATourneyCompactResults.csv')
    .pipe(process_games)
)

# Read in submission games
df_sub = (
    pd.read_csv(data_raw_dir + 'SampleSubmissionStage1.csv')
    .pipe(process_submission_games)
)

In [9]:
# display_df(df_seeds, title="Seeds")
# display_df(df_games, title="Games")
# display_df(df_sub, title="SubmissionGames")

# Merge Datasets

In [12]:
def merge_seed_dataset(df, df_seeds, submission_file):
    """TBD"""
    df = (
        df
        .merge(df_seeds.rename(index=str, columns={'TeamID': 'TeamOneID'}), on=['Season', 'TeamOneID'])
        .rename(index=str, columns={'Seed': 'TeamOneSeed', 'IsPlayIn': 'TeamOneIsPlayIn'})
        .merge(df_seeds.rename(index=str, columns={'TeamID': 'TeamTwoID'}), on=['Season', 'TeamTwoID'])
        .rename(index=str, columns={'Seed': 'TeamTwoSeed', 'IsPlayIn': 'TeamTwoIsPlayIn'})  
    )
    
    if not(submission_file):
        df = df.loc[~(df.TeamOneIsPlayIn & df.TeamTwoIsPlayIn)]  # remove play-in games 
    
    return df.drop(['TeamOneIsPlayIn', 'TeamTwoIsPlayIn'], axis=1)

In [13]:
# Merge tournament games with seeds
df_interim_mdl = (
    df_games
    .pipe(merge_seed_dataset, df_seeds=df_seeds, submission_file=False)
)

# Merge submission games with seeds
df_interim_sub = (
    df_sub
    .pipe(merge_seed_dataset, df_seeds=df_seeds, submission_file=True)
)

In [18]:
# display_df(df_interim_mdl)
# display_df(df_interim_sub)

# Create Features

In [15]:
def create_seedDiff_feat(df):
    return (
        df
        .assign(SeedDiff=df.TeamOneSeed - df.TeamTwoSeed)
    )

In [16]:
# Create features for modeling dataset
df_interim_mdl = (
    df_interim_mdl
    .pipe(create_seedDiff_feat)
)

# Create features for submission dataset
df_interim_sub = (
    df_interim_sub
    .pipe(create_seedDiff_feat)
)

In [19]:
display_df(df_interim_mdl, n=2, title="Games")
display_df(df_interim_sub, n=2, title="Submissin Games")

Games:


,Season,Label,TeamOneID,TeamTwoID,TeamOneSeed,TeamTwoSeed,SeedDiff
0,1985,1,1116,1234,9,8,1
1,1985,0,1116,1385,9,1,8


,Season,Label,TeamOneID,TeamTwoID,TeamOneSeed,TeamTwoSeed,SeedDiff
2182,2018,1,1243,1420,9,16,-7
2183,2018,0,1243,1260,9,11,-2


(2142, 7)

Submissin Games:


,Season,TeamOneID,TeamTwoID,TeamOneSeed,TeamTwoSeed,SeedDiff
0,2014,1107,1110,16,15,1
1,2014,1107,1112,16,1,15


,Season,TeamOneID,TeamTwoID,TeamOneSeed,TeamTwoSeed,SeedDiff
11388,2018,1455,1462,4,1,3
11389,2018,1460,1462,14,1,13


(11390, 6)

In [20]:
# Save datasets
df_interim_mdl.to_csv(data_interim_dir + 'model_dataset.csv', index=False)
df_interim_sub.to_csv(data_interim_dir + 'submission_dataset.csv', index=False)